# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902077


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/88 [00:00<?, ?it/s]

Rendering models:   2%|▏         | 2/88 [00:01<01:12,  1.18it/s]

Rendering models:   3%|▎         | 3/88 [00:05<02:21,  1.66s/it]

Rendering models:   5%|▍         | 4/88 [00:05<01:40,  1.20s/it]

Rendering models:   6%|▌         | 5/88 [00:07<01:51,  1.34s/it]

...

Rendering models:  92%|█████████▏| 81/88 [00:14<00:00, 11.08it/s]

Rendering models:  94%|█████████▍| 83/88 [00:14<00:00, 12.17it/s]

Rendering models:  98%|█████████▊| 86/88 [00:14<00:00, 13.29it/s]

Rendering models: 100%|██████████| 88/88 [00:14<00:00, 14.48it/s]

kayleebug2017                         0.001895
JessieDawn                            0.075392
not-logged-in-5ab67c604320a0edd4c1    0.000680
not-logged-in-0b0736c8991a7cc2017a    0.001157
Wain-PalmerM                          0.002263
not-logged-in-8c0c44faf754d344b18a    0.021061
Thedunkmasta                          0.002593
DrJackie                              0.001706
klangon88                             0.001105
Thedunkmasta                          0.004879
msalin03                              0.000836
clizardo                              0.001081
clizardo                              0.001902
mkirms                                0.000657
msross                                0.000587
M_Mij                                 0.000590
gfess                                 0.006664
acohen12                              0.001106
mchaffin                              0.000432
djcameron                             0.000621
stgomes                               0.029350
wthammas     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())